# OpenEO Connection to VITO Backend

In [1]:
import openeo
import logging
from openeo.auth.auth_bearer import BearerAuth
logging.basicConfig(level=logging.INFO)

In [2]:
# Define constants

# Connection
VITO_DRIVER_URL = "http://openeo.vgt.vito.be/openeo/0.4.0"
OUTPUT_FILE = "/tmp/openeo_vito_output.tiff"
OUTFORMAT = "tiff"

# Data
PRODUCT_ID = "BIOPAR_FAPAR_V1_GLOBAL"

DATE_START = "2016-01-01T00:00:00Z"
DATE_END = "2016-03-10T23:59:59Z"

IMAGE_WEST = 16.138916
IMAGE_EAST = 16.524124
IMAGE_NORTH = 48.320647
IMAGE_SOUTH = 48.138600
IMAGE_SRS = "EPSG:4326"

# Processes
NDVI_RED = "B4"
NDVI_NIR = "B8A"

STRECH_COLORS_MIN = -1
STRECH_COLORS_MAX = 1

In [3]:
# Connect with VITO backend
connection = openeo.connect(VITO_DRIVER_URL)
connection

In [4]:
# Get available processes from the backend.
processes = connection.list_processes()
processes

[{'description': 'Aggregates zonal statistics for one or multiple polygons over the spatial dimensions.\n\nThe process considers all pixels for which the point at the pixel center intersects with the corresponding polygon (as defined in the Simple Features standard by the OGC).\n\nThe data cube must have been reduced to only contain two raster dimensions and a third dimension the values are aggregated for, for example the temporal dimension to get a time series. Otherwise this process fails with the `TooManyDimensions` error.\n\nThe number of total and valid pixels is returned together with the calculated values.',
  'name': 'aggregate_polygon',
  'parameters': {'binary': {'description': 'Specifies whether the process should pass two values to the reducer or a list of values (default).\n\nIf the process passes two values, the reducer must be both associative and commutative as the execution may be executed in parallel and therefore the order of execution is arbitrary.\n\nThis parameter

In [5]:
# Retrieve the list of available collections
collections = connection.list_collections()

list(collections)[:2]

[{'bands': [{'band_id': 'classification',
    'name': 'classification',
    'res_m': 20,
    'type': 'int8',
    'unit': '1'}],
  'description': 'Sen2Cor Scene Classification layer, from Sentinel 2 data.',
  'extent': {'crs': 'EPSG:3857',
   'spatial': [180, -56, -180, 83],
   'temporal': ['2015-07-06', None]},
  'id': 'S2_FAPAR_SCENECLASSIFICATION_V102_PYRAMID',
  'license': 'free',
  'links': [{'href': 'https://terrascope.be/en/products',
    'rel': 'alternate',
    'title': 'Product web page.'}],
  'name': 'S2_FAPAR_SCENECLASSIFICATION_V102_PYRAMID',
  'product_id': 'S2_FAPAR_SCENECLASSIFICATION_V102_PYRAMID',
  'source': 'Terrascope - VITO',
  'title': 'Sen2Cor Scene Classification layer'},
 {'bands': [],
  'description': 'Sentinel 2 Level-3: fCOVER, web mercator projection, with overviews.',
  'extent': {'crs': 'EPSG:3857',
   'spatial': [180, -56, -180, 83],
   'temporal': ['2015-07-06', None]},
  'id': 'S2_FCOVER_PYRAMID',
  'license': 'free',
  'links': [],
  'name': 'S2_FCOVER

In [6]:
# Get detailed information about a collection
process = connection.describe_collection(PRODUCT_ID)
process

{'bands': [],
 'data_id': 'BIOPAR_FAPAR_V1_GLOBAL',
 'description': 'The FAPAR quantifies the fraction of the solar radiation absorbed by live leaves for the photosynthesis activity. Then, it refers only to the green and alive elements of the canopy. The FAPAR depends on the canopy structure, vegetation element optical properties, atmospheric conditions, and angular configuration. To overcome this latter dependency, a daily integrated FAPAR value is assessed.\n\nFAPAR is recognized as an Essential Climate Variable (ECV) by the Global Climate Observing System (GCOS).',
 'extent': {'crs': 'EPSG:4326',
  'spatial': [180, -56, -180, 83],
  'temporal': ['2014-01-01', None]},
 'id': 'BIOPAR_FAPAR_V1_GLOBAL',
 'license': 'free',
 'links': [{'href': 'https://land.copernicus.eu/global/products/fapar',
   'rel': 'alternate',
   'title': 'Product web page.'}],
 'name': 'BIOPAR_FAPAR_V1_GLOBAL',
 'product_id': 'BIOPAR_FAPAR_V1_GLOBAL',
 'title': 'Copernicus Global Land FAPAR product V1'}

In [7]:
# Select collection product
datacube = connection.imagecollection(PRODUCT_ID)

print(datacube.to_json())

{'getcollection1': {'arguments': {'name': 'BIOPAR_FAPAR_V1_GLOBAL'},
  'process_id': 'get_collection',
  'result': False}}

In [8]:
# Specifying the date range and the bounding box
datacube = datacube.filter_bbox(west=IMAGE_WEST, east=IMAGE_EAST, north=IMAGE_NORTH,
                                             south=IMAGE_SOUTH, crs=IMAGE_SRS)
datacube = datacube.filter_daterange(extent=[DATE_START, DATE_END])

print(datacube.to_json())

{'filterbbox1': {'arguments': {'crs': 'EPSG:4326',
   'data': {'from_node': 'getcollection1'},
   'east': 16.524124,
   'north': 48.320647,
   'south': 48.1386,
   'west': 16.138916},
  'process_id': 'filter_bbox',
  'result': False},
 'filtertemporal1': {'arguments': {'data': {'from_node': 'filterbbox1'},
   'from': '2016-01-01T00:00:00Z',
   'to': '2016-03-10T23:59:59Z'},
  'process_id': 'filter_temporal',
  'result': False},
 'getcollection1': {'arguments': {'name': 'BIOPAR_FAPAR_V1_GLOBAL'},
  'process_id': 'get_collection',
  'result': False}}

In [9]:
# Applying some operations on the data
datacube = datacube.ndvi(red=NDVI_RED, nir=NDVI_NIR)
datacube = datacube.min_time()

print(datacube.to_json())

{'NDVI1': {'arguments': {'data': {'from_node': 'filtertemporal1'}},
  'process_id': 'NDVI',
  'result': False},
 'filterbbox1': {'arguments': {'crs': 'EPSG:4326',
   'data': {'from_node': 'getcollection1'},
   'east': 16.524124,
   'north': 48.320647,
   'south': 48.1386,
   'west': 16.138916},
  'process_id': 'filter_bbox',
  'result': False},
 'filtertemporal1': {'arguments': {'data': {'from_node': 'filterbbox1'},
   'from': '2016-01-01T00:00:00Z',
   'to': '2016-03-10T23:59:59Z'},
  'process_id': 'filter_temporal',
  'result': False},
 'getcollection1': {'arguments': {'name': 'BIOPAR_FAPAR_V1_GLOBAL'},
  'process_id': 'get_collection',
  'result': False},
 'reduce1': {'arguments': {'data': {'from_node': 'NDVI1'},
   'dimension': 'temporal',
   'reducer': {'callback': {'r1': {'arguments': {'data': {'from_argument': 'data'}},
      'process_id': 'min',
      'result': True}}}},
  'process_id': 'reduce',
  'result': False}}

In [10]:
# Sending the job to the backend
job = datacube.create_job()
job.start_job()
job

In [29]:
# Describe Job
job.describe_job()

{'job_id': 'af23e66a-fe85-4a0f-87a4-de8ec00e7f33', 'status': 'finished'}

In [30]:
# Download job result
job.download_results(OUTPUT_FILE)

job

In [32]:
# Showing the result
from IPython.display import Image
result = Image(filename=OUTPUT_FILE)

result

#from PIL import Image
#resp2 = req.get(OUTPUT_FILE)
#resp2.raw.decode_content = True
#im = Image.open(resp2.raw)
#im


ValueError: Cannot embed the 'tiff' image format